# Init

In [1]:
library(tidyverse)
library(readxl)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## True and Sensor Data Import for Calibration

In [2]:
#True Data import

smdata_raw <- read.csv('data/true_data/nmin_raw.csv', sep = ';', na.strings = c("#VALUE!", "NA", "#DIV/0!"))

smdata <- smdata_raw %>%
    mutate(date = dmy(Date),
           depth = recode(Depth,
                         '0-30' = 30,
                         '30-60' = 60,
                         '60-90' = 90,
                         .default = NA_real_)) %>% 
    select(-c(Date, Depth, H2O_wet)) %>% 
    group_by(patchID, date, depth) %>% 
    summarise(across(where(is.double), ~ mean(.x, na.rm = TRUE))) %>% 
    mutate(vwc = BD * H2O_dry) %>% 
    rename(patch = patchID, gwc = H2O_dry)

write_csv(smdata, file = "data/true_data/nmin.csv")

`summarise()` has grouped output by 'patchID', 'date'. You can override using
the `.groups` argument.


In [3]:
# # Sensor Data single date import

sensor_smdata <- read.csv("data/dailydata/all_data_daily.csv", 
                        sep = ";")

# sensor_single_dates <- dmy(c('11-Nov-20','2-Feb-21','10-Dec-21','28-Feb-22'))

# sensor_single_smdata <- sensor_smdata %>% 
#     mutate(date = ymd(datetime),
#        mean_30 = (water_content_right_30_cm + water_content_left_30_cm) / 2,
#        mean_60 = (water_content_right_60_cm + water_content_left_60_cm) / 2,
#        mean_90 = (water_content_right_90_cm + water_content_left_90_cm) / 2) %>% 
#     select(patch, date, contains('water_content')) %>% 
#     filter(date %in% sensor_single_dates) 

In [4]:
dates <- smdata %>% 
    group_by(patch) %>% 
    reframe(dates = as.Date(as.POSIXct(unique(date))))

dates_list <- with(dates, split(dates, patch))

patches <- smdata %>% 
    group_by(date) %>% 
    reframe(patches = unique(patch))

patches_list <- with(patches, split(patches, date))

In [5]:
# Sensor Data import

sensor_data_temp <- sensor_smdata %>% 
    mutate(date = ymd(datetime),
       mean_30 = (water_content_right_30_cm + water_content_left_30_cm) / 2,
       mean_60 = (water_content_right_60_cm + water_content_left_60_cm) / 2,
       mean_90 = (water_content_right_90_cm + water_content_left_90_cm) / 2) %>% 
    select(patch, date, contains('water_content')) 

sensor_smdata <- data.frame()

# Iterate over each item in the data list
for (date_str in names(patches_list)) {
  # Convert the date string to Date format
  dates <- ymd(date_str)
  # Get the patches for this date from the data list
  patches <- patches_list[[date_str]]
  # Filter the rows of the data frame where the date is this date and the patch is in these patches
  filtered_rows <- sensor_data_temp %>% 
    filter(date == dates & patch %in% patches)
  # Append the filtered rows to the filtered data frame
  sensor_smdata <- bind_rows(sensor_smdata, filtered_rows)
}

sensor_smdata_l <- sensor_smdata %>%
    pivot_longer(cols=-c(date,patch),
                 cols_vary = 'slowest',
                 names_to = c('.value', 'depth'),
                 names_pattern = "water_content_(.*)_(.*)_cm",
                 names_transform = list(
                     depth = ~ readr::parse_double(.x)))

In [6]:
# Combine True and sensor data

smdata_f <- sensor_smdata_l  %>% left_join(smdata, by = c("patch", "date", "depth"))

## Calibration Model

In [7]:
# Define the list of predictor variables
predictors <- c("left", "right")

# Initialize an empty list to store the models
cal_models <- list()
cal_models_sir_list <- list()
cal_models_sir_df <- data.frame(depth = integer(),
                                side = factor(),
                                patch = integer(), 
                                slope = double(), 
                                intercept = double(), 
                                rsq = double(),
                                n = integer())


# Outer loop over the predictors
for (pred in predictors) {

    # Define the formula as a string, then convert it to a formula object
    formula_obj <- as.formula(paste("vwc ~", pred))

    # Fit the models using by(), and add them to the model_list
    models <- by(smdata_f,
                list(as.factor(smdata_f$depth), as.factor(smdata_f$patch)),
                function(subset) lm(formula_obj, data = subset))

    # Convert by object to list
    models_list <- as.list(models)

    # Loop over depth indices
    for (i in 1:dim(models)[1]) {
      # Loop over patch indices
      for (j in 1:dim(models)[2]) {
        # Check if the model for this combination exists
        if (!is.null(models_list[[i,j]])) {
          # Get the depth and patch indices from the by object
          depth_index <- as.character(attr(models, "dimnames")[[1]][i])
          patch_index <- as.character(attr(models, "dimnames")[[2]][j])
          # Use these indices in the cal_models
          cal_models[[paste0(pred, "_", depth_index, "_", patch_index)]] <- summary(models_list[[i,j]])
          cal_models_sir_list[[paste0(pred, "_", depth_index, "_", patch_index)]]$int <- models_list[[i,j]]$coefficients[[1]]
          cal_models_sir_list[[paste0(pred, "_", depth_index, "_", patch_index)]]$slope <- models_list[[i,j]]$coefficients[[2]]
          cal_models_sir_list[[paste0(pred, "_", depth_index, "_", patch_index)]]$rsq <- summary(models_list[[i,j]])$r.squared
          sir <- data.frame(depth = as.integer(depth_index), 
                            side = as.factor(pred),
                            patch = as.integer(patch_index), 
                            slope = as.double(models_list[[i,j]]$coefficients[[2]]),
                            intercept = as.double(models_list[[i,j]]$coefficients[[1]]),
                            rsq = summary(models_list[[i,j]])$r.squared,
                            n = nobs(models_list[[i,j]]))

          cal_models_sir_df <- rbind(cal_models_sir_df, sir)
        }
      }
    }
}

In [8]:
smdata_cal <- cal_models_sir_df %>% 
    select(-c(rsq, n)) %>% 
    pivot_wider(names_from = side, 
                values_from = c(slope, intercept), 
                names_glue = "{.value}_{side}") %>% 
    left_join(smdata_f, by = c("patch",  "depth")) %>% 
    rename("right_pre" = "right",
           "left_pre" = "left")%>% 
    mutate("left_post" = (slope_left * left_pre) + intercept_left,
           "right_post" = (slope_right * right_pre) + intercept_right) %>%
    distinct()

In [9]:
summary(cal_models_sir_df)

     depth       side        patch            slope          intercept       
 Min.   :30   left :90   Min.   : 12.00   Min.   :0.2921   Min.   :-14.7741  
 1st Qu.:30   right:90   1st Qu.: 49.00   1st Qu.:0.7799   1st Qu.: -0.7724  
 Median :60              Median : 67.00   Median :0.9922   Median :  2.5448  
 Mean   :60              Mean   : 67.63   Mean   :1.1035   Mean   :  2.4396  
 3rd Qu.:90              3rd Qu.: 95.00   3rd Qu.:1.2684   3rd Qu.:  5.1685  
 Max.   :90              Max.   :119.00   Max.   :3.6961   Max.   : 16.2065  
      rsq               n        
 Min.   :0.2828   Min.   :3.000  
 1st Qu.:0.7217   1st Qu.:5.000  
 Median :0.8498   Median :6.000  
 Mean   :0.7994   Mean   :5.878  
 3rd Qu.:0.9250   3rd Qu.:7.000  
 Max.   :0.9934   Max.   :8.000  

## Applying Calibration to Cleaned Data

In [10]:
clean_csvs <- list.files(path = "data/sensor_data_clean", full.names = TRUE)

In [11]:
# Function to calibrate data

calibrator <- cal_models_sir_df %>% 
    select(c(depth, side, patch, slope, intercept))

calibrate <- function(file_path) {

    df <- read.csv(file_path)

    patch_num <- as.integer(str_extract(file_path, "\\d+"))

    relevent_cal <- calibrator %>% 
        filter(patch == patch_num) %>% 
        right_join(df, by = c("depth", "side", "patch")) %>% 
        mutate(cal_wc = (wc * slope) + intercept) %>% 
        select(c(dateTime, depth, patch, side, cal_wc))

    output_file_path <- str_replace(file_path, "sensor_data_clean", "sensor_data_clean_cal")
    
    write_csv(relevent_cal, file = output_file_path)
}

lapply(clean_csvs, calibrate)

: 

In [2]:
library(tidyverse)
library(readxl)

In [10]:
# Create all_sensor_data_clean_cal

fp = "data/sensor_data_clean_cal"

clean_cal_csvs <- list.files(path = fp, full.names = TRUE)
merge_list <- lapply(clean_cal_csvs, read.csv)
all_sdcc <- do.call("rbind", merge_list)

write_csv(all_sdcc, file = paste0(fp, "/all_sensor_data_clean_cal.csv"))

## Plotting Calibrated and Uncalibrated

In [12]:
smplots_depth<- ggplot(smdata_f, aes(y=vwc, color=factor(patch))) + 
    geom_abline(slope = 1, intercept = 0) +
    geom_point(aes(x=left), shape = 4)+
    geom_smooth(aes(x=left), method='lm', se = FALSE)+
    geom_point(aes(x=right), shape = 1)+
    geom_smooth(aes(x=right), method='lm',se = FALSE, )+
    scale_color_discrete()+
    labs(x='SENSOR SM', y = 'TRUE SM')+
    facet_wrap(vars(patch,depth))

smplots<- ggplot(smdata_f, aes(y=vwc, color=factor(patch))) + 
    geom_abline(slope = 1, intercept = 0) +
    geom_point(aes(x=left), shape = 4)+
    geom_smooth(aes(x=left, linetype = factor(depth)), method='lm', se = FALSE)+
    geom_point(aes(x=right), shape = 1)+
    geom_smooth(aes(x=right, linetype = factor(depth)), method='lm',se = FALSE, )+
    scale_color_discrete()+
    labs(x='SENSOR SM', y = 'TRUE SM')+
    facet_wrap(vars(patch))

ggsave('temp/smplots_depths.tiff', smplots_depth, width=20, height =20, dpi=300, limitsize=FALSE) 
ggsave('temp/smplots.tiff', smplots, width=10, height =10, dpi=300, limitsize=FALSE) 

`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."
`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."


In [13]:
smplots_depth_cal<- ggplot(smdata_cal, aes(y=vwc, color=factor(patch))) + 
    geom_abline(slope = 1, intercept = 0) +
    geom_point(aes(x=left_post), shape = 4)+
    geom_smooth(aes(x=left_post), method='lm', se = FALSE)+
    geom_point(aes(x=right_post), shape = 1)+
    geom_smooth(aes(x=right_post), method='lm',se = FALSE, )+
    scale_color_discrete()+
    labs(x='SENSOR SM', y = 'TRUE SM')+
    facet_wrap(vars(patch,depth))

smplots_cal<- ggplot(smdata_cal, aes(y=vwc, color=factor(patch))) + 
    geom_abline(slope = 1, intercept = 0) +
    geom_point(aes(x=left_post), shape = 4)+
    geom_smooth(aes(x=left_post, linetype = factor(depth)), method='lm', se = FALSE)+
    geom_point(aes(x=right_post), shape = 1)+
    geom_smooth(aes(x=right_post, linetype = factor(depth)), method='lm',se = FALSE, )+
    scale_color_discrete()+
    labs(x='SENSOR SM', y = 'TRUE SM')+
    facet_wrap(vars(patch))

ggsave('temp/smplots_depths_cal.tiff', smplots_depth_cal, width=20, height =20, dpi=300, limitsize=FALSE) 
ggsave('temp/smplots_cal.tiff', smplots_cal, width=10, height =10, dpi=300, limitsize=FALSE) 

`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."
`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
`geom_smooth()` using formula = 'y ~ x'
Warning message:
"Removed 134 rows containing non-finite values (`stat_smooth()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 134 rows containing missing values (`geom_point()`)."
